In [ ]:
!pip install nltk
!pip install tokenizers
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ce5891af26c94999c7f1bebb495f257e512526fca00e94710a9bbf98a3beac98
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

***Preparing Base Data - Hugging Face***



In [ ]:
questions_df_1 = pd.read_csv("/content/drive/MyDrive/NLP/QA_Dataset.csv")
questions_df_1.rename(columns={'question': 'Question', 'answer': 'Answer'}, inplace=True)
questions_df_2 = pd.read_csv('/content/drive/MyDrive/NLP/NLP_Project_Dataset_v1_400.csv')

eval_df = pd.concat([questions_df_1, questions_df_2])

In [ ]:
hf_df = pd.read_csv('/content/drive/MyDrive/NLP/Gen_Ans_Dataset/GPT_HuggingFace_Full.csv')

In [ ]:
hf_df = hf_df[['Answer','answer-records-200-OpenAPI', 'answer-ner-200-OpenAPI', 'answer-pos-200-OpenAPI', 'answer-all_tags-200-OpenAPI',
  'answer-records-400-OpenAPI', 'answer-ner-400-OpenAPI','answer-pos-400-OpenAPI',  'answer-all_tags-400-OpenAPI',
  'answer-records-600-OpenAPI', 'answer-ner-600-OpenAPI', 'answer-pos-600-OpenAPI', 'answer-all_tags-600-OpenAPI']]

In [ ]:
hf_df.columns = [col.replace('OpenAPI', 'HF_GPT') for col in hf_df.columns]

In [ ]:
hf_df.columns

Index(['Answer', 'answer-records-200-HF_GPT', 'answer-ner-200-HF_GPT',
       'answer-pos-200-HF_GPT', 'answer-all_tags-200-HF_GPT',
       'answer-records-400-HF_GPT', 'answer-ner-400-HF_GPT',
       'answer-pos-400-HF_GPT', 'answer-all_tags-400-HF_GPT',
       'answer-records-600-HF_GPT', 'answer-ner-600-HF_GPT',
       'answer-pos-600-HF_GPT', 'answer-all_tags-600-HF_GPT'],
      dtype='object')

***BLEU Score Calculation***

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# Function to calculate BLEU scores with smoothing for multiple columns
def calculate_bleu_scores_multiple(df, reference_col, candidate_cols):
    results = {}
    smoothing = SmoothingFunction().method7  # Using smoothing method 1
    for candidate_col in candidate_cols:
        scores = []
        for index, row in df.iterrows():
            # Convert to string and handle NaNs
            reference_text = str(row[reference_col]) if pd.notna(row[reference_col]) else ''
            candidate_text = str(row[candidate_col]) if pd.notna(row[candidate_col]) else ''

            # Tokenizing and converting to lower case
            reference = word_tokenize(reference_text.lower())
            candidate = word_tokenize(candidate_text.lower())

            # Calculate BLEU score with smoothing
            score = sentence_bleu([reference], candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)
            scores.append(score)
        results[f'BLEU_score_{candidate_col}'] = scores
    return pd.DataFrame(results)

#Calculating BLEU scores for multiple columns
bleu_eval_df = calculate_bleu_scores_multiple(hf_df, 'Answer', candidate_cols)

In [ ]:
import pandas as pd
from rouge_score import rouge_scorer
from sklearn.metrics import precision_recall_fscore_support
from nltk.tokenize import word_tokenize

# Function to calculate ROUGE scores
def calculate_rouge_scores(df, reference_col, candidate_cols):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = {}
    for candidate_col in candidate_cols:
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []
        for index, row in df.iterrows():
            ref = str(row[reference_col]) if pd.notna(row[reference_col]) else ''
            cand = str(row[candidate_col]) if pd.notna(row[candidate_col]) else ''
            scores = scorer.score(ref, cand)
            rouge1_scores.append(scores['rouge1'].fmeasure)
            rouge2_scores.append(scores['rouge2'].fmeasure)
            rougeL_scores.append(scores['rougeL'].fmeasure)

        results[f'{candidate_col}_rouge1'] = rouge1_scores
        results[f'{candidate_col}_rouge2'] = rouge2_scores
        results[f'{candidate_col}_rougeL'] = rougeL_scores

    # Convert results dictionary into DataFrame
    return pd.DataFrame(results)

#Running the function and printing the results
rouge_results = calculate_rouge_scores(hf_df, 'Answer', candidate_cols)

In [ ]:
import pandas as pd
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Function to calculate METEOR scores for multiple columns with proper tokenization
def calculate_meteor_scores(df, reference_col, candidate_cols):
    results = {}
    for candidate_col in candidate_cols:
        scores = []
        for index, row in df.iterrows():
            # Handle NaN values and ensure data is in string format, then tokenize
            ref = word_tokenize(str(row[reference_col]) if pd.notna(row[reference_col]) else '')
            cand = word_tokenize(str(row[candidate_col]) if pd.notna(row[candidate_col]) else '')
            # Calculate METEOR score using tokenized texts
            score = meteor_score([ref], cand)
            scores.append(score)
        results[f'METEOR_score_{candidate_col}'] = scores

    return pd.DataFrame(results)

meteor_results = calculate_meteor_scores(hf_df, 'Answer', candidate_cols)

In [ ]:
def calculate_mean_scores(*dataframes):
    mean_scores = {}
    for df in dataframes:
        for column in df.columns:
            mean_scores[column] = df[column].mean()
    return mean_scores

# Calculating mean scores for both DataFrames
# mean_scores = calculate_mean_scores(rouge_results, bleu_eval_df, meteor_results)

***Preparing Base Data - OpenAI***

In [ ]:
openai_df = pd.read_csv('/content/drive/MyDrive/NLP/Gen_Ans_Dataset/GPT_OpenAI_AllChunks_AnsOnly.csv')
openai_df = openai_df[['Answer', 'answer-records-200-OpenAPI',
       'answer-ner-200-OpenAPI', 'answer-pos-200-OpenAPI',
       'answer-all_tags-200-OpenAPI', 'answer-records-400-OpenAPI',
       'answer-ner-400-OpenAPI', 'answer-pos-400-OpenAPI',
       'answer-all_tags-400-OpenAPI', 'answer-records-600-OpenAPI',
       'answer-ner-600-OpenAPI', 'answer-pos-600-OpenAPI',
       'answer-all_tags-600-OpenAPI']]

In [ ]:
candidate_cols = ['answer-records-200-OpenAPI', 'answer-ner-200-OpenAPI', 'answer-pos-200-OpenAPI', 'answer-all_tags-200-OpenAPI',
                  'answer-records-400-OpenAPI', 'answer-ner-400-OpenAPI', 'answer-pos-400-OpenAPI', 'answer-all_tags-400-OpenAPI',
                  'answer-records-600-OpenAPI', 'answer-ner-600-OpenAPI', 'answer-pos-600-OpenAPI', 'answer-all_tags-600-OpenAPI']

In [ ]:
bleu_eval_openai = calculate_bleu_scores_multiple(openai_df, 'Answer', candidate_cols)
rouge_openai = calculate_rouge_scores(openai_df, 'Answer', candidate_cols)
meteor_openai = calculate_meteor_scores(openai_df, 'Answer', candidate_cols)

In [ ]:
mean_scores_openai = calculate_mean_scores(rouge_openai, bleu_eval_openai, meteor_openai)
print(mean_scores_openai)

{'answer-records-200-OpenAPI_rouge1': 0.3658768387683259, 'answer-records-200-OpenAPI_rouge2': 0.186904305278999, 'answer-records-200-OpenAPI_rougeL': 0.33235358484887034, 'answer-ner-200-OpenAPI_rouge1': 0.3899056192339038, 'answer-ner-200-OpenAPI_rouge2': 0.18446495163506055, 'answer-ner-200-OpenAPI_rougeL': 0.35772719886273097, 'answer-pos-200-OpenAPI_rouge1': 0.4163651852069032, 'answer-pos-200-OpenAPI_rouge2': 0.17646916096458215, 'answer-pos-200-OpenAPI_rougeL': 0.3857815334422763, 'answer-all_tags-200-OpenAPI_rouge1': 0.38891343193380495, 'answer-all_tags-200-OpenAPI_rouge2': 0.1790245521844671, 'answer-all_tags-200-OpenAPI_rougeL': 0.3587204463688809, 'answer-records-400-OpenAPI_rouge1': 0.39220250686898184, 'answer-records-400-OpenAPI_rouge2': 0.19880454452013427, 'answer-records-400-OpenAPI_rougeL': 0.3604462121465178, 'answer-ner-400-OpenAPI_rouge1': 0.3895539773584549, 'answer-ner-400-OpenAPI_rouge2': 0.18380465533714885, 'answer-ner-400-OpenAPI_rougeL': 0.3569722263130557,

***SBERT - Evaluation metric***


- Trial

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
def preprocess_text(text):
  text = text.lower().strip()
  translator = str.maketrans('', '', string.punctuation)
  text = text.translate(translator)
  return text

ground_truth = preprocess_text("Apple is expected to release a new iPhone this year.")
generated_response = preprocess_text("According to rumors, Apple might launch a new iPhone in the fall.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
ground_truth_embedding = model.encode(ground_truth)
generated_response_embedding = model.encode(generated_response)

In [ ]:
# Cosine Similarity for sentence embeddings
similarity = util.cos_sim(ground_truth_embedding, generated_response_embedding)
print(f"Similarity Score: {similarity.squeeze()}")  # Squeeze removes extra dimension

Similarity Score: 0.8956094980239868


- Hugging Face Model


In [ ]:
hf_candidate_cols = ['answer-records-200-HF_GPT', 'answer-ner-200-HF_GPT',
       'answer-pos-200-HF_GPT', 'answer-all_tags-200-HF_GPT',
       'answer-records-400-HF_GPT', 'answer-ner-400-HF_GPT',
       'answer-pos-400-HF_GPT', 'answer-all_tags-400-HF_GPT',
       'answer-records-600-HF_GPT', 'answer-ner-600-HF_GPT',
       'answer-pos-600-HF_GPT', 'answer-all_tags-600-HF_GPT']

In [ ]:
def evaluate_responses_with_sbert(df, sbert_model_name='distilbert-base-nli-mean-tokens'):
    model = SentenceTransformer(sbert_model_name)
    def evaluate_single_response(row):
        ground_truth = row['Answer']
        for col in candidate_cols:
            response = row[col]
            embeddings = model.encode([ground_truth, response])
            similarity = util.cos_sim(embeddings[0], embeddings[1])
            new_col_name = col + '_SBERT'
            row[new_col_name] = similarity.item()  # Store similarity
        return row

    return df.apply(evaluate_single_response, axis=1)

In [ ]:
sbert_eval_df = evaluate_responses_with_sbert(hf_df)

NameError: name 'SentenceTransformer' is not defined

In [ ]:
sbert_eval_df = sbert_eval_df[['answer-records-200-HF_GPT_SBERT', 'answer-ner-200-HF_GPT_SBERT',
       'answer-pos-200-HF_GPT_SBERT', 'answer-all_tags-200-HF_GPT_SBERT',
       'answer-records-400-HF_GPT_SBERT', 'answer-ner-400-HF_GPT_SBERT',
       'answer-pos-400-HF_GPT_SBERT', 'answer-all_tags-400-HF_GPT_SBERT',
       'answer-records-600-HF_GPT_SBERT', 'answer-ner-600-HF_GPT_SBERT',
       'answer-pos-600-HF_GPT_SBERT', 'answer-all_tags-600-HF_GPT_SBERT']]
sbert_eval_df.head()

,answer-records-200-HF_GPT_SBERT,answer-ner-200-HF_GPT_SBERT,answer-pos-200-HF_GPT_SBERT,answer-all_tags-200-HF_GPT_SBERT,answer-records-400-HF_GPT_SBERT,answer-ner-400-HF_GPT_SBERT,answer-pos-400-HF_GPT_SBERT,answer-all_tags-400-HF_GPT_SBERT,answer-records-600-HF_GPT_SBERT,answer-ner-600-HF_GPT_SBERT,answer-pos-600-HF_GPT_SBERT,answer-all_tags-600-HF_GPT_SBERT
0,0.718391,0.812522,0.718391,0.792109,0.800278,0.792109,0.792109,0.792109,0.800278,0.792109,0.792109,0.800278
1,0.717178,0.663292,0.728448,0.771287,0.771287,0.771287,0.771287,0.771287,0.541770,0.465325,0.476643,0.516471
2,0.804368,0.814033,0.823120,0.827578,0.826370,0.785114,0.815408,0.786762,0.770019,0.777913,0.754316,0.786762
3,0.511639,0.440103,0.428595,0.562780,0.434603,0.459793,0.427388,0.399996,0.462539,0.445195,0.376215,0.403156
4,0.787373,0.753962,0.861851,0.627687,0.813659,0.811065,0.597798,0.627687,0.813659,0.811065,0.597798,0.627687


In [ ]:
sbert_mean = calculate_mean_scores(sbert_eval_df)

- Open AI


In [ ]:
candidate_cols

['answer-records-200-OpenAPI',
 'answer-ner-200-OpenAPI',
 'answer-pos-200-OpenAPI',
 'answer-all_tags-200-OpenAPI',
 'answer-records-400-OpenAPI',
 'answer-ner-400-OpenAPI',
 'answer-pos-400-OpenAPI',
 'answer-all_tags-400-OpenAPI',
 'answer-records-600-OpenAPI',
 'answer-ner-600-OpenAPI',
 'answer-pos-600-OpenAPI',
 'answer-all_tags-600-OpenAPI']

In [ ]:
sbert_openai_eval_df = evaluate_responses_with_sbert(openai_df)

In [ ]:
sbert_openai_eval_df_copy = sbert_openai_eval_df

In [ ]:
sbert_openai_eval_df_copy = sbert_openai_eval_df_copy[['answer-records-200-OpenAPI_SBERT', 'answer-ner-200-OpenAPI_SBERT',
       'answer-pos-200-OpenAPI_SBERT', 'answer-all_tags-200-OpenAPI_SBERT',
       'answer-records-400-OpenAPI_SBERT', 'answer-ner-400-OpenAPI_SBERT',
       'answer-pos-400-OpenAPI_SBERT', 'answer-all_tags-400-OpenAPI_SBERT',
       'answer-records-600-OpenAPI_SBERT', 'answer-ner-600-OpenAPI_SBERT',
       'answer-pos-600-OpenAPI_SBERT', 'answer-all_tags-600-OpenAPI_SBERT']]

In [ ]:
mean_openai_sbert = calculate_mean_scores(sbert_openai_eval_df_copy)

In [ ]:
print(mean_openai_sbert)

{'answer-records-200-OpenAPI_SBERT': 0.6665119894611584, 'answer-ner-200-OpenAPI_SBERT': 0.6839939983573133, 'answer-pos-200-OpenAPI_SBERT': 0.7125853240574277, 'answer-all_tags-200-OpenAPI_SBERT': 0.6793985283050165, 'answer-records-400-OpenAPI_SBERT': 0.674252897648547, 'answer-ner-400-OpenAPI_SBERT': 0.6762205024169367, 'answer-pos-400-OpenAPI_SBERT': 0.715334747910285, 'answer-all_tags-400-OpenAPI_SBERT': 0.6675646245199803, 'answer-records-600-OpenAPI_SBERT': 0.6924678248127178, 'answer-ner-600-OpenAPI_SBERT': 0.6945603282129538, 'answer-pos-600-OpenAPI_SBERT': 0.7494238202111343, 'answer-all_tags-600-OpenAPI_SBERT': 0.6956527718791256}


***BERTSCORE _ SAMPLE***

In [ ]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [ ]:
from bert_score import score

# Lists of reference texts and candidate texts
references = ["the cat sits outside", "a man is playing a guitar"]
candidates = ["the cat is sitting out", "a man is playing guitar"]

# Calculate BERTScore
P, R, F1 = score(candidates, references, lang="en", verbose=True)

# Print the results
print("Precision: ", P)
print("Recall: ", R)
print("F1 Score: ", F1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.13 seconds, 0.94 sentences/sec
Precision:  tensor([0.9442, 0.9863])
Recall:  tensor([0.9513, 0.9793])
F1 Score:  tensor([0.9477, 0.9828])


In [ ]:
import pandas as pd
from bert_score import score

def calculate_bert_scores(df, reference_col, candidate_cols):
    results = {}
    for candidate_col in candidate_cols:
        # Extracting the lists of reference and candidate texts
        references = [str(x) if pd.notna(x) else '' for x in df[reference_col]]
        candidates = [str(x) if pd.notna(x) else '' for x in df[candidate_col]]

        # Calculating BERTScore
        P, R, F1 = score(candidates, references, lang="en", verbose=True)

        # Storing the results
        results[candidate_col + '_precision'] = P.numpy().tolist()
        results[candidate_col + '_recall'] = R.numpy().tolist()
        results[candidate_col + '_f1'] = F1.numpy().tolist()

    # Converting results to DataFrame for easier handling
    result_df = pd.DataFrame(results)
    return result_df

In [ ]:
bert_score_hf_df = calculate_bert_scores(hf_df, 'Answer', hf_candidate_cols)

In [ ]:
sbert_eval_df.to_csv('/content/drive/MyDrive/NLP/sbert_eval_df', index=False)
sbert_openai_eval_df.to_csv('/content/drive/MyDrive/NLP/sbert_openai_eval_df', index=False)
bert_score_hf_df.to_csv('/content/drive/MyDrive/NLP/bert_score_hf_df', index=False)
bleu_eval_openai.to_csv('/content/drive/MyDrive/NLP/bleu_eval_openai', index=False)
rouge_openai.to_csv('/content/drive/MyDrive/NLP/rouge_openai', index=False)
meteor_openai.to_csv('/content/drive/MyDrive/NLP/meteor_openai', index=False)


In [ ]:
bleu_eval_df.to_csv('/content/drive/MyDrive/NLP/bleu_eval_df', index=False)
rouge_results.to_csv('/content/drive/MyDrive/NLP/rouge_results', index=False)
meteor_results.to_csv('/content/drive/MyDrive/NLP/meteor_results', index=False)

In [ ]:
bert_score_hf_df = pd.read_csv('/content/drive/MyDrive/NLP/bert_score_hf_df')

In [ ]:
bert_Score_Mean = calculate_mean_scores(bert_score_hf_df)

In [ ]:
bert_Score_Mean

{'answer-records-200-HF_GPT_precision': 0.8977941732102461,
 'answer-records-200-HF_GPT_recall': 0.9253383519227613,
 'answer-records-200-HF_GPT_f1': 0.9105621397249983,
 'answer-ner-200-HF_GPT_precision': 0.8973231114477778,
 'answer-ner-200-HF_GPT_recall': 0.9192467699816198,
 'answer-ner-200-HF_GPT_f1': 0.9074852702058391,
 'answer-pos-200-HF_GPT_precision': 0.9000331850208864,
 'answer-pos-200-HF_GPT_recall': 0.9140860377025212,
 'answer-pos-200-HF_GPT_f1': 0.906459090641006,
 'answer-all_tags-200-HF_GPT_precision': 0.8873602340741412,
 'answer-all_tags-200-HF_GPT_recall': 0.9078185623320042,
 'answer-all_tags-200-HF_GPT_f1': 0.8967456400639726,
 'answer-records-400-HF_GPT_precision': 0.8903854430457692,
 'answer-records-400-HF_GPT_recall': 0.913569708665212,
 'answer-records-400-HF_GPT_f1': 0.9010353009887193,
 'answer-ner-400-HF_GPT_precision': 0.8916803007263215,
 'answer-ner-400-HF_GPT_recall': 0.9135830804644298,
 'answer-ner-400-HF_GPT_f1': 0.9018078274687622,
 'answer-pos-40

In [ ]:
bert_score_openai_df = calculate_bert_scores(openai_df, 'Answer', candidate_cols)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/14 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/13 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 313.30 seconds, 1.55 sentences/sec


In [ ]:
bert_score_openai_df.to_csv('/content/drive/MyDrive/NLP/bert_score_openai_df', index=False)

In [ ]:
calculate_mean_scores(bert_score_openai_df)

{'answer-records-200-OpenAPI_precision': 0.89050336016549,
 'answer-records-200-OpenAPI_recall': 0.9137983058460455,
 'answer-records-200-OpenAPI_f1': 0.901168760080887,
 'answer-ner-200-OpenAPI_precision': 0.8910311713140198,
 'answer-ner-200-OpenAPI_recall': 0.9123435729325063,
 'answer-ner-200-OpenAPI_f1': 0.9008242214412846,
 'answer-pos-200-OpenAPI_precision': 0.89566975071597,
 'answer-pos-200-OpenAPI_recall': 0.9102295851756516,
 'answer-pos-200-OpenAPI_f1': 0.9022725203645573,
 'answer-all_tags-200-OpenAPI_precision': 0.8920782400992672,
 'answer-all_tags-200-OpenAPI_recall': 0.9111322588881348,
 'answer-all_tags-200-OpenAPI_f1': 0.9007575763351142,
 'answer-records-400-OpenAPI_precision': 0.890968965895382,
 'answer-records-400-OpenAPI_recall': 0.9162529440818991,
 'answer-records-400-OpenAPI_f1': 0.9026307363078427,
 'answer-ner-400-OpenAPI_precision': 0.8859472327516893,
 'answer-ner-400-OpenAPI_recall': 0.9098307388554875,
 'answer-ner-400-OpenAPI_f1': 0.896963431020823,
 '

-> Preprocess LLAMA txt files

In [ ]:
file_path1 = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/RAGanswers200HGFLlama.txt'
file_path2 = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/RAGanswers200HGFLlama400Ques.txt'
output_file_path = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/LLAMA-200-HF_Combined.txt.txt'

with open(file_path1, 'r', encoding='utf-8') as file1:
    data1 = file1.read()
with open(file_path2, 'r', encoding='utf-8') as file2:
    data2 = file2.read()

with open(output_file_path, 'w', encoding='utf-8') as outfile:
    outfile.write(data1)
    outfile.write(data2)

In [ ]:
file_path1 = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/RAGanswers400HGFLlama.txt'
file_path2 = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/RAGanswers400HGFLlama400Ques.txt'
output_file_path = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/LLAMA-400-HF_Combined.txt.'

def read_file_safe(path, encodings=['utf-8', 'windows-1252']):
    for encoding in encodings:
        try:
            with open(path, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            continue
    raise ValueError(f'All specified encodings failed for file: {path}')

# Read the contents of the files with a fallback for encoding issues
data1 = read_file_safe(file_path1)
data2 = read_file_safe(file_path2)

# Open the output file in write mode and write both sets of data to it
with open(output_file_path, 'w', encoding='utf-8') as outfile:
    # Write the contents of the first file
    outfile.write(data1)
    # Write the contents of the second file
    outfile.write(data2)

ValueError: All specified encodings failed for file: /content/drive/MyDrive/NLP/LLAMA_Preprocessing/RAGanswers400HGFLlama400Ques.txt

In [ ]:
def convert_file_encoding(source_file_path, output_file_path):
    # Read the content of the source file with the specified source encoding
    with open(source_file_path, 'r', encoding='windows-1252') as file:
        content = file.read()

    # Write the content to a new file with the desired target encoding
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(content)

In [ ]:
convert_file_encoding()

In [ ]:
llama_hf_df = eval_df

In [ ]:
import pandas as pd
import re

file_path = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/LLAMA-200-HF_Combined.txt.txt'

with open(file_path, 'r', encoding='utf-8') as file:
  data = file.read()
  answers = re.findall(r'Answer:\s*(.*?)\s*Source Docs:', data, re.DOTALL)
  cleaned_answers = [re.sub(r'^.*?\n', '', answer.strip()) for answer in answers]
  llama_hf_df['answer-llama-200-hf'] = cleaned_answers

In [ ]:
import pandas as pd
import re

file_path = '/content/drive/MyDrive/NLP/LLAMA_Preprocessing/LLAMA-400-HF_Combined.txt.'

with open(file_path, 'r', encoding='utf-8') as file:
  data = file.read()
  answers = re.findall(r'Answer:\s*(.*?)\s*Source Docs:', data, re.DOTALL)
  cleaned_answers = [re.sub(r'^.*?\n', '', answer.strip()) for answer in answers]
  llama_hf_df['answer-llama-400-hf'] = cleaned_answers

ValueError: Length of values (404) does not match length of index (486)